In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

In [ ]:
output_path = '/data'

# places_k = pd.read_parquet(f"{output_path}/stay_locations_k_places.parquet")
places_k = pd.read_parquet(f"{output_path}/places_k.parquet")

## Calculate $q_K$

In [ ]:
def calculate_qk_alignment(places_df, user_col='user_id', k_freq_col='k_freq', k_dist_col='k_dist'):
    """
    Calculate qK alignment using both Jaccard similarityfor each user
    
    This function:
    1. Classifies each place into categories based on K-freq and K-dist indicators
    2. Calculates Jaccard similarity for each user
    3. Returns user-level alignment metrics and place categorizations
    
    Parameters:
    -----------
    places_df : DataFrame
        Stay locations dataframe with user_id and K-place indicators
    user_col : str
        Column name for user identifier
    k_freq_col : str
        Column name for K-freq indicator (0 or 1)
    k_dist_col : str
        Column name for K-dist indicator (0 or 1)
    
    Returns:
    --------
    tuple : (user_alignment_df, places_with_categories_df)
        - user_alignment_df: User-level qK metrics
        - places_with_categories_df: Original dataframe with place categories added
    """
    
    print(f"🔄 CALCULATING qK ALIGNMENT METRICS")
    print("=" * 60)
    
    # Create a copy to avoid modifying original data
    places_analysis = places_df.copy()
    
    # Ensure K-place indicators are binary (0 or 1)
    places_analysis[k_freq_col] = places_analysis[k_freq_col].fillna(0).astype(int)
    places_analysis[k_dist_col] = places_analysis[k_dist_col].fillna(0).astype(int)
    
    print(f"Total places: {len(places_analysis)}")
    print(f"Users: {places_analysis[user_col].nunique()}")
    
    # Step 1: Assign place categories based on K-freq and K-dist values
    def get_place_category(row):
        k_freq = row[k_freq_col]
        k_dist = row[k_dist_col]
        
        if k_freq == 1 and k_dist == 1:
            return 'f1d1'  # Both methods identify this place
        elif k_freq == 1 and k_dist == 0:
            return 'f1d0'  # Only K-freq identifies this place
        elif k_freq == 0 and k_dist == 1:
            return 'f0d1'  # Only K-dist identifies this place
        elif k_freq == 0 and k_dist == 0:
            return 'f0d0'  # Neither method identifies this place
        else:
            return 'other'
    
    places_analysis['k_type'] = places_analysis.apply(get_place_category, axis=1)
    
    # Step 2: Aggregate by user to count places in each category
    user_place_counts = places_analysis.groupby(user_col)['k_type'].value_counts().unstack(fill_value=0)

    # Ensure all categories exist in the dataframe
    for category in ['f1d1', 'f1d0', 'f0d1', 'f0d0']:
        if category not in user_place_counts.columns:
            user_place_counts[category] = 0
    
    user_place_counts = user_place_counts.reset_index()
    
    print(f"Place category distribution:")
    for category in ['f1d1', 'f1d0', 'f0d1', 'f0d0']:
        total_places = user_place_counts[category].sum()
        print(f"  {category}: {total_places} places")
    
    # Step 3: Calculate Jaccard similarity for each user
    def calculate_jaccard(row):
        """Jaccard = |A ∩ B| / |A ∪ B| = f1d1 / (f1d1 + f1d0 + f0d1)"""
        numerator = row['f1d1']
        denominator = row['f1d1'] + row['f1d0'] + row['f0d1']
        return numerator / denominator if denominator > 0 else 0.0
    
    # Calculate alignment metrics
    user_place_counts['jaccard'] = user_place_counts.apply(calculate_jaccard, axis=1)
    
    # Calculate additional metrics
    user_place_counts['total_k_places'] = user_place_counts['f1d1'] + user_place_counts['f1d0'] + user_place_counts['f0d1']
    user_place_counts['total_places'] = user_place_counts['f1d1'] + user_place_counts['f1d0'] + user_place_counts['f0d1'] + user_place_counts['f0d0']
    
    # Calculate ratios
    user_place_counts['f1d1_ratio'] = user_place_counts['f1d1'] / user_place_counts['total_places']
    user_place_counts['f1d0_ratio'] = user_place_counts['f1d0'] / user_place_counts['total_places']
    user_place_counts['f0d1_ratio'] = user_place_counts['f0d1'] / user_place_counts['total_places']
    user_place_counts['f0d0_ratio'] = user_place_counts['f0d0'] / user_place_counts['total_places']
    
    # Calculate K-method coverage
    user_place_counts['k_freq_coverage'] = (user_place_counts['f1d1'] + user_place_counts['f1d0']) / user_place_counts['total_places']
    user_place_counts['k_dist_coverage'] = (user_place_counts['f1d1'] + user_place_counts['f0d1']) / user_place_counts['total_places']
    
    print(f"\n ALIGNMENT SUMMARY STATISTICS")
    print("=" * 40)
    print(f"Users analyzed: {len(user_place_counts)}")
    print(f"Jaccard similarity - Mean: {user_place_counts['jaccard'].mean():.4f}, Median: {user_place_counts['jaccard'].median():.4f}")
    

    return user_place_counts, places_analysis

In [ ]:
# Calculate qK alignment using the no-work variants
user_qk, places_with_categories = calculate_qk_alignment(
    places_df=places_k,
    user_col='user_id', 
    k_freq_col='k_freq',  # Using no-work version
    k_dist_col='k_dist'   # Using no-work version
)

### Aggregating at grid level

In [ ]:
# Read the grid data
grid_df = gpd.read_parquet(
    'grid_data'
)

In [ ]:
grid_qk = user_qk.groupby(['grd_id']).agg({
    # general metrics
    'user_id': 'nunique',

    # qK metrics
    'jaccard': 'mean',

    'jaccard_nw': 'mean',
    
}).reset_index()

grid_analysis = grid_qk.merge(
    grid_df[['grd_id', 'city', 'population']],
    left_on='grd_id',
    right_on='grd_id',
    how='left'
)

## $q_K$ null model
The null model fixed distance-visitation probability for each user, but randomly shuffle the visitations.

In [ ]:
# Step 1: Extract overall distance-frequency pattern using binning approach
print("🔄 EXTRACTING OVERALL DISTANCE-FREQUENCY PATTERN WITH EQUAL BINNING")
print("=" * 60)

# Use your existing data - places_k contains the mobility data
places_data = places_k

# Filter for valid data
valid_data = places_data[
    (places_data['home_dist'].notna()) & 
    (places_data['visit_freq'] >= 2) &
    (places_data['home_dist'] > 0)
].copy()

print(f"Valid records for analysis: {len(valid_data)}")

# Extract distance and frequency for all users combined
distances = valid_data['home_dist'].values
frequencies = valid_data['visit_freq'].values

# Log-transform for power law fitting
log_dist = np.log10(distances)
log_freq = np.log10(frequencies)

# Create equally spaced bins for log-transformed data (20 equal pieces)
n_bins = 20

# Equal spacing in log space for distances
log_dist_min, log_dist_max = log_dist.min(), log_dist.max()
log_distance_bins = np.linspace(log_dist_min, log_dist_max, n_bins + 1)

# Equal spacing in log space for frequencies  
log_freq_min, log_freq_max = log_freq.min(), log_freq.max()
log_frequency_bins = np.linspace(log_freq_min, log_freq_max, n_bins + 1)

print(f"Using {n_bins} equal bins")
print(f"Log distance range: {log_dist_min:.3f} to {log_dist_max:.3f}")
print(f"Log frequency range: {log_freq_min:.3f} to {log_freq_max:.3f}")

# Assign each observation to distance bins (using log distances)
valid_data['log_distance_bin'] = pd.cut(log_dist, bins=log_distance_bins, include_lowest=True)

# Calculate bin statistics using equal spacing
bin_stats = valid_data.groupby('log_distance_bin', observed=True).agg({
    'home_dist': ['mean', 'median', 'count'],
    'visit_freq': ['mean', 'median', 'std']
}).round(3)

# Flatten column names
bin_stats.columns = ['_'.join(col).strip() for col in bin_stats.columns]
bin_stats = bin_stats.reset_index()

# Filter bins with sufficient observations (optional)
min_obs_per_bin = 10
bin_stats_filtered = bin_stats[bin_stats['home_dist_count'] >= min_obs_per_bin].copy()

print(f"Bins after filtering (≥{min_obs_per_bin} obs): {len(bin_stats_filtered)}")

# Prepare data for regression on bin means
bin_distances = bin_stats_filtered['home_dist_mean'].values
bin_frequencies = bin_stats_filtered['visit_freq_mean'].values
bin_weights = bin_stats_filtered['home_dist_count'].values  # Weight by number of observations

# Log-transform bin means for power law fitting
log_bin_dist = np.log10(bin_distances)
log_bin_freq = np.log10(bin_frequencies)

# Fit power law: freq = a * dist^b using weighted linear regression on binned data
try:
    from scipy import stats
    from sklearn.linear_model import LinearRegression
    
    # Option 1: Simple regression (unweighted)
    slope, intercept, r_value, p_value, std_err = stats.linregress(log_bin_dist, log_bin_freq)
    
    # Option 2: Weighted regression (weighted by bin size)
    weighted_reg = LinearRegression()
    weighted_reg.fit(log_bin_dist.reshape(-1, 1), log_bin_freq, sample_weight=bin_weights)
    weighted_slope = weighted_reg.coef_[0]
    weighted_intercept = weighted_reg.intercept_
    
    # Calculate R² for weighted regression
    y_pred_weighted = weighted_reg.predict(log_bin_dist.reshape(-1, 1))
    weighted_r_squared = 1 - np.average((log_bin_freq - y_pred_weighted)**2, weights=bin_weights) / np.average((log_bin_freq - np.average(log_bin_freq, weights=bin_weights))**2, weights=bin_weights)
    
    # Store model parameters (using weighted version)
    overall_model = {
        'slope': weighted_slope,
        'intercept': weighted_intercept,
        'r_squared': weighted_r_squared,
        'n_bins': len(bin_stats_filtered),
        'n_observations': bin_weights.sum(),
        'unweighted_slope': slope,
        'unweighted_r_squared': r_value**2
    }
    
    print(f"Binned model (weighted): slope={weighted_slope:.3f}, R²={weighted_r_squared:.3f}")
    print(f"Binned model (unweighted): slope={slope:.3f}, R²={r_value**2:.3f}")
    print(f"Number of bins used: {len(bin_stats_filtered)}")
    print(f"Total observations: {bin_weights.sum()}")
    
    # Optional: Plot the binned data and fit
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 6))
    
    # Plot bin means with error bars (weighted by bin size)
    plt.scatter(bin_distances, bin_frequencies, s=10, alpha=0.7, label='Bin means (equal spacing)')
    
    # Plot fitted line
    x_pred = np.linspace(bin_distances.min(), bin_distances.max(), 100)
    y_pred = 10**(weighted_intercept) * x_pred**weighted_slope
    plt.plot(x_pred, y_pred, 'r-', label=f'Fit: freq = {10**weighted_intercept:.2f} × dist^{weighted_slope:.3f}')
    
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Distance (m)')
    plt.ylabel('Visit Frequency')
    plt.title(f'Equal-Binned Distance-Frequency Relationship (R² = {weighted_r_squared:.3f})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()
    
except Exception as e:
    print(f"Error fitting binned model: {e}")
    # Use default parameters if fitting fails
    overall_model = {
        'slope': -1.0,  # Default negative slope
        'intercept': 0.0,
        'n_bins': 0,
        'n_observations': len(valid_data)
    }

print(f"Final model parameters: slope={overall_model['slope']:.3f}, intercept={overall_model['intercept']:.3f}")

# # Save bin statistics for further analysis
# bin_stats_filtered.to_csv(f"{output_path}/distance_frequency_equal_bins.csv", index=False)

In [ ]:
# Step 2: Create distance-based probability function
print("\n🎲 CREATING DISTANCE-BASED PROBABILITY FUNCTION")
print("=" * 60)

def calculate_distance_probability(distances):
    """Calculate selection probability based on distance using empirical slope"""
    # Avoid division by zero
    distances = np.maximum(distances, 1.0)
    
    # Use empirical slope (usually negative) to weight by distance
    weights = distances ** overall_model['slope']
    
    # Normalize to probabilities
    probabilities = weights / weights.sum()
    
    return probabilities

print(f"Distance-based probability function created with slope: {overall_model['slope']:.3f}")

In [ ]:
# Step 3: Prepare data for K-rand generation
print("\n📍 PREPARING DATA FOR K-RAND GENERATION")
print("=" * 60)

amenity_list = [
    'CIVIC_RELIGION', 'CULTURE', 'DINING', 'EDUCATION', 'FITNESS', 'GROCERIES', 'HEALTHCARE', 'RETAIL', 'SERVICE', 'TRANSPORT'
    ]


smallest_values = np.ones(len(amenity_list), dtype=int)

# Get all available places for each user
all_places = places_data[['user_id', 'stay_gid10', 'visit_freq', 'home_dist', 'home_gid9', 'work_gid9','name'] + amenity_list].copy()

# Get user data with their K-dist places for reference
user_k_dist_data = places_data[places_data['k_dist'] == 1].copy()

print(f"Total available places: {len(all_places)}")
print(f"K-dist places: {len(user_k_dist_data)}")
print(f"Users with K-dist places: {user_k_dist_data['user_id'].nunique()}")

# Get unique users for processing
unique_users = user_k_dist_data['user_id'].unique()
print(f"Users to process: {len(unique_users)}")

In [ ]:
def select_users_for_qk_rand(user_qk, fraction=0.1, min_users_per_grid=1, max_users_per_grid=100, 
                            selected_cities=None, random_state=42):
    """
    Select a representative subset of users for qK rand calculation based on grid cells from specific cities
    
    Parameters:
    -----------
    user_qk : DataFrame
        User-level data with 'user_id' and 'name' columns
    fraction : float
        Fraction of users to select from each grid (default: 0.1 for 10%)
    min_users_per_grid : int
        Minimum number of users to select per grid (if available)
    max_users_per_grid : int
        Maximum number of users to select per grid
    selected_cities : list, optional
        List of city names to select users from. If None, uses all cities.
    random_state : int
        Random seed for reproducibility
    
    Returns:
    --------
    tuple : (selected_user_ids, selection_stats_df)
        - selected_user_ids: List of selected user IDs for qK rand calculation
        - selection_stats_df: DataFrame with selection statistics by city
    """
    
    print(f"🎯 SELECTING USERS FOR qK RAND CALCULATION")
    print("=" * 60)
    
    # Set random seed for reproducibility
    np.random.seed(random_state)
    
    # Filter for selected cities if specified
    if selected_cities is not None:
        print(f"Filtering for cities: {selected_cities}")
        user_qk_filtered = user_qk[user_qk['name'].isin(selected_cities)].copy()
        cities_found = user_qk_filtered['name'].unique()
        cities_missing = set(selected_cities) - set(cities_found)
        
        if cities_missing:
            print(f"⚠️  Cities not found in data: {list(cities_missing)}")
        print(f"✅ Cities found: {list(cities_found)}")
    else:
        user_qk_filtered = user_qk.copy()
        print("Using all cities in dataset")
    
    if len(user_qk_filtered) == 0:
        print("❌ No users found in selected cities!")
        return [], pd.DataFrame()
    
    # Group users by city
    city_user_counts = user_qk_filtered.groupby('name')['user_id'].nunique().reset_index()
    city_user_counts.columns = ['name', 'total_users']
    
    print(f"Total cities: {len(city_user_counts)}")
    print(f"Total users in selected cities: {user_qk_filtered['user_id'].nunique()}")
    print(f"Target selection fraction: {fraction:.1%}")
    
    selected_users = []
    city_selection_stats = []
    
    # Process each city
    for city_name, city_data in user_qk_filtered.groupby('name'):
        city_users = city_data['user_id'].unique()
        n_users_in_city = len(city_users)
        
        # Calculate target number of users to select from this city
        target_users = max(
            min_users_per_grid,
            min(max_users_per_grid, int(n_users_in_city * fraction))
        )
        
        # Don't select more users than available
        target_users = min(target_users, n_users_in_city)
        
        # Randomly sample users from this city
        if target_users > 0:
            selected_city_users = np.random.choice(
                city_users, 
                size=target_users, 
                replace=False
            ).tolist()
            
            selected_users.extend(selected_city_users)
            
            city_selection_stats.append({
                'name': city_name,
                'total_users': n_users_in_city,
                'selected_users': target_users,
                'selection_rate': target_users / n_users_in_city
            })
    
    # Create summary statistics
    selection_stats_df = pd.DataFrame(city_selection_stats)
    
    total_selected = len(selected_users)
    total_users_filtered = user_qk_filtered['user_id'].nunique()
    overall_rate = total_selected / total_users_filtered if total_users_filtered > 0 else 0
    
    print(f"\n📊 SELECTION SUMMARY:")
    print(f"Users selected: {total_selected:,}")
    print(f"Overall selection rate: {overall_rate:.1%}")
    if len(selection_stats_df) > 0:
        print(f"Average users per city: {selection_stats_df['selected_users'].mean():.1f}")
    print(f"Cities processed: {len(selection_stats_df)}")
    
    # Show city-level breakdown
    if len(selection_stats_df) > 0:
        print(f"\n🏙️ CITY-LEVEL BREAKDOWN:")
        print(f"{'City':<15} {'Total':<8} {'Selected':<10} {'Rate':<8}")
        print("-" * 45)
        for _, row in selection_stats_df.iterrows():
            print(f"{row['name']:<15} {row['total_users']:<8} {row['selected_users']:<10} {row['selection_rate']:<8.1%}")
    
    return selected_users, selection_stats_df

# Example usage:
# Select users from specific major cities
major_cities = ['Helsinki', 'Espoo', 'Tampere', 'Vantaa', 'Oulu', 'Turku','Jyväskylä','Kuopio', 'Lahti', 'Pori','Joensuu','Kouvola']

selected_users, selection_stats = select_users_for_qk_rand(
    user_qk,
    fraction=0.5,  # 50% selection rate
    min_users_per_grid=1,  # At least 5 users per city
    max_users_per_grid=500,  # Maximum 200 users per city
    selected_cities=major_cities,
    random_state=42
)

print(f"\n✅ Selected {len(selected_users)} users for qK rand calculation from {len(major_cities)} cities")

In [ ]:
# Step 4-6: Generate Multiple K-rand Scenarios Per User and Calculate Average qK

print("\n🎯 GENERATING MULTIPLE K-RAND SCENARIOS PER USER")
print("=" * 70)

def select_k_rand_places_for_user(user_id, user_places, required_amenities, smallest_values):
    """
    Select K-rand places for a user following K-visitation framework:
    Randomly select places until amenity requirements are met
    """
    # Calculate selection probabilities based on distance
    distances = user_places['home_dist'].values
    probabilities = calculate_distance_probability(distances)
    
    # Check for invalid probabilities and handle edge cases
    if np.any(np.isnan(probabilities)) or np.any(np.isinf(probabilities)) or probabilities.sum() == 0:
        # Fall back to uniform probabilities if calculation fails
        probabilities = np.ones(len(user_places)) / len(user_places)
    
    # Initialize tracking
    selected_places = []
    cumulative_amenities = np.zeros(len(required_amenities))
    available_indices = list(range(len(user_places)))
    
    # Continue selecting until requirements are met or no more places available
    max_iterations = min(len(user_places), 50)  # Prevent infinite loops
    
    for iteration in range(max_iterations):
        # Check if requirements are met
        if np.all(cumulative_amenities >= smallest_values):
            break
            
        # If no more places available, break
        if len(available_indices) == 0:
            break
        
        # Select next place based on distance probabilities
        # Update probabilities for remaining places
        remaining_probs = probabilities[available_indices]
        
        # Ensure probabilities are valid and sum to 1
        if remaining_probs.sum() == 0 or np.any(np.isnan(remaining_probs)) or np.any(np.isinf(remaining_probs)):
            # Use uniform probabilities if issues detected
            remaining_probs = np.ones(len(available_indices)) / len(available_indices)
        else:
            remaining_probs = remaining_probs / remaining_probs.sum()  # Renormalize
        
        try:
            selected_idx = np.random.choice(
                available_indices,
                size=1,
                p=remaining_probs
            )[0]
        except (ValueError, TypeError) as e:
            # If probability calculation still fails, select randomly without probabilities
            print(f"Warning: Probability calculation failed for user {user_id}, using uniform selection: {e}")
            selected_idx = np.random.choice(available_indices)
        
        # Add selected place
        selected_place = user_places.iloc[selected_idx].copy()
        selected_places.append(selected_place)
        
        # Update cumulative amenities
        place_amenities = selected_place[required_amenities].astype(float).values
        cumulative_amenities += place_amenities
        
        # Remove selected place from available options
        available_indices.remove(selected_idx)
    
    if len(selected_places) == 0:
        return None
    
    # Create result DataFrame
    result_df = pd.DataFrame(selected_places)
    result_df['k_rand'] = 1
    result_df['user_id'] = user_id
    
    # Determine completion status
    requirements_met = np.all(cumulative_amenities >= smallest_values)
    result_df['k_rand_status'] = 'complete' if requirements_met else 'incomplete'
    
    return result_df
def calculate_user_qk_rand(user_id, user_places, user_k_dist_data, amenity_list, smallest_values, n_iterations=50):
    """
    Generate multiple K-rand scenarios for a single user and calculate average qK-rand
    
    Parameters:
    -----------
    user_id : str
        User identifier
    user_places : DataFrame
        Available places for this user
    user_k_dist_data : DataFrame
        User's K-dist places for reference
    amenity_list : list
        List of amenity column names
    smallest_values : array
        Minimum required values for each amenity
    n_iterations : int
        Number of K-rand scenarios to generate
    
    Returns:
    --------
    dict : User-level results with average qK metrics
    """
    
    if len(user_places) == 0 or len(user_k_dist_data) == 0:
        return None
    
    # Get user's K-dist places for comparison
    k_dist_places = set(user_k_dist_data['stay_gid10'])
    
    qk_results = []
    
    for iteration in range(n_iterations):
        try:
            # Generate K-rand places for this iteration
            user_k_rand = select_k_rand_places_for_user(
                user_id, 
                user_places, 
                amenity_list, 
                smallest_values
            )
            
            if user_k_rand is not None:
                # Get K-rand places for this iteration
                k_rand_places = set(user_k_rand['stay_gid10'])
                
                # Calculate overlaps
                f1d1 = len(k_rand_places & k_dist_places)  # Both methods
                f1d0 = len(k_rand_places - k_dist_places)  # Only K-rand
                f0d1 = len(k_dist_places - k_rand_places)  # Only K-dist
                
                # Calculate Jaccard similarity (qK)
                union_size = f1d1 + f1d0 + f0d1
                jaccard_rand = f1d1 / union_size if union_size > 0 else 0
                
                # Calculate Dice coefficient
                dice_rand = (2 * f1d1) / (2 * f1d1 + f1d0 + f0d1) if (2 * f1d1 + f1d0 + f0d1) > 0 else 0
                
                qk_results.append({
                    'jaccard_rand': jaccard_rand,
                    'dice_rand': dice_rand,
                    'f1d1': f1d1,
                    'f1d0': f1d0,
                    'f0d1': f0d1,
                    'k_rand_count': len(user_k_rand),
                    'k_rand_status': user_k_rand['k_rand_status'].iloc[0]
                })
                
        except Exception as e:
            # Skip this iteration if it fails
            continue
    
    # Calculate averages across all successful iterations
    if len(qk_results) > 0:
        avg_result = {
            'user_id': user_id,
            'n_iterations_success': len(qk_results),
            'jaccard_rand_mean': np.mean([r['jaccard_rand'] for r in qk_results]),
            'jaccard_rand_std': np.std([r['jaccard_rand'] for r in qk_results]),
            'dice_rand_mean': np.mean([r['dice_rand'] for r in qk_results]),
            'dice_rand_std': np.std([r['dice_rand'] for r in qk_results]),
            'f1d1_mean': np.mean([r['f1d1'] for r in qk_results]),
            'f1d0_mean': np.mean([r['f1d0'] for r in qk_results]),
            'f0d1_mean': np.mean([r['f0d1'] for r in qk_results]),
            'k_rand_count_mean': np.mean([r['k_rand_count'] for r in qk_results]),
            'k_dist_count': len(k_dist_places),
            'complete_rate': np.mean([1 if r['k_rand_status'] == 'complete' else 0 for r in qk_results])
        }
        
        return avg_result
    
    return None

# Process each user with multiple iterations
print(f"Processing {len(selected_users)} users with multiple K-rand scenarios each...")

user_qk_rand_results = []
n_iterations_per_user = 10  # Number of K-rand scenarios per user

for user_id in tqdm(selected_users, desc="Processing users"):

    # Get available places for this user
    user_places = all_places[all_places['user_id'] == user_id].copy()
    # print(user_places)

    # Get user's K-dist data
    user_k_dist = user_k_dist_data[
        (user_k_dist_data['user_id'] == user_id) &
        (user_k_dist_data['k_dist'] == 1)
    ]
    
    if len(user_places) == 0 or len(user_k_dist) == 0:
        continue
    
    # Calculate average qK-rand for this user
    user_result = calculate_user_qk_rand(
        user_id, 
        user_places, 
        user_k_dist,
        amenity_list, 
        smallest_values, 
        n_iterations=n_iterations_per_user
    )
    
    # print(user_result)

    if user_result is not None:
        user_qk_rand_results.append(user_result)

# Convert to DataFrame
user_qk_rand_df = pd.DataFrame(user_qk_rand_results)

print(f"✅ User-level qK-rand calculated for {len(user_qk_rand_df)} users")

# Display summary statistics
print(f"\n📊 USER-LEVEL QK-RAND SUMMARY:")
print("=" * 40)
print(f"Mean Jaccard (rand): {user_qk_rand_df['jaccard_rand_mean'].mean():.4f}")
print(f"Mean Dice (rand): {user_qk_rand_df['dice_rand_mean'].mean():.4f}")
print(f"Average completion rate: {user_qk_rand_df['complete_rate'].mean():.3f}")

# # Show variability statistics
# print(f"\n📈 VARIABILITY STATISTICS:")
# print(f"Average Jaccard std per user: {user_qk_rand_df['jaccard_rand_std'].mean():.4f}")
# print(f"Average Dice std per user: {user_qk_rand_df['dice_rand_std'].mean():.4f}")

user_qk_rand_df.head()